Last updated: 05/11/22 by ivanp@us.ibm.com

# Watson Natural Language Understanding

## Load the ibm-watson libraries

In [5]:
pip install ibm-watson

Note: you may need to restart the kernel to use updated packages.


In [6]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, CategoriesOptions, KeywordsOptions, SentimentOptions

In [7]:
#pip3 install pandas
import pandas as pd
import json


## Load the blogs data asset as a Pandas dataframe

In [8]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d77c4db6ce1d438e92a1f00e7f2d5d2d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='hDekfN2FWkKS_xY6ChfWhFXk4iptKdhTP50fnLoIltjt',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_d77c4db6ce1d438e92a1f00e7f2d5d2d.get_object(Bucket='graphai-donotdelete-pr-4nvni6nug5azgz',Key='ibm-blogs.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

blogs_df = pd.read_csv(body)
blogs_df.head()


,id,title,link,date
0,1001,Cloud at the Edge,https://www.ibm.com/cloud/blog/cloud-at-the-edge,2/26/19
1,1002,Rounding out the Edges,https://www.ibm.com/cloud/blog/rounding-out-th...,5/7/19
2,1003,Architecting at the Edge,https://www.ibm.com/cloud/blog/architecting-at...,10/21/19
3,1004,DevOps at the Edge,https://www.ibm.com/cloud/blog/devops-at-the-edge,12/3/19
4,1005,Policies at the Edge,https://www.ibm.com/cloud/blog/policies-at-the...,1/22/20


In [9]:
#Load ibm-blogs.csv

#blogs_df = pd.read_csv("ibm-blogs.csv")
print("Number of rows in ibm-blogs file: ", blogs_df.shape[0])
blogs_df.head()

Number of rows in ibm-blogs file:  21


,id,title,link,date
0,1001,Cloud at the Edge,https://www.ibm.com/cloud/blog/cloud-at-the-edge,2/26/19
1,1002,Rounding out the Edges,https://www.ibm.com/cloud/blog/rounding-out-th...,5/7/19
2,1003,Architecting at the Edge,https://www.ibm.com/cloud/blog/architecting-at...,10/21/19
3,1004,DevOps at the Edge,https://www.ibm.com/cloud/blog/devops-at-the-edge,12/3/19
4,1005,Policies at the Edge,https://www.ibm.com/cloud/blog/policies-at-the...,1/22/20


## Create a Watson NLU Instance to get your IAM_API key

In [9]:
IAM_KEY = '{API_KEY}'
SERVICE_URL = 'https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/{INSTANCE_ID}'

In [10]:
authenticator = IAMAuthenticator(IAM_KEY)
natural_language_understanding = NaturalLanguageUnderstandingV1(version='2020-08-01',authenticator=authenticator)

natural_language_understanding.set_service_url(SERVICE_URL)

In [15]:
def get_NLU_Response(link):
    response = natural_language_understanding.analyze(
    url = link,
    #features=Features(categories=CategoriesOptions(limit=3))).get_result()
    features=Features(entities=EntitiesOptions(sentiment=False,limit=3))).get_result()
    
    return response

In [12]:
response=get_NLU_Response(blogs_df['link'][0])

In [14]:
print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 4285,
    "features": 1
  },
  "retrieved_url": "https://www.ibm.com/cloud/blog/cloud-at-the-edge",
  "language": "en",
  "categories": [
    {
      "score": 0.869217,
      "label": "/technology and computing"
    },
    {
      "score": 0.823215,
      "label": "/technology and computing/internet technology"
    },
    {
      "score": 0.80841,
      "label": "/technology and computing/networking/network monitoring and management"
    }
  ]
}


In [16]:
response=get_NLU_Response(blogs_df['link'][1])

In [17]:
print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 6652,
    "features": 1
  },
  "retrieved_url": "https://www.ibm.com/cloud/blog/rounding-out-the-edges",
  "language": "en",
  "entities": [
    {
      "type": "Company",
      "text": "NFV",
      "relevance": 0.95577,
      "count": 2,
      "confidence": 0.557096
    },
    {
      "type": "Company",
      "text": "Edge Cloud",
      "relevance": 0.720689,
      "count": 1,
      "confidence": 0.230761
    },
    {
      "type": "Company",
      "text": "NaaS",
      "relevance": 0.425252,
      "count": 1,
      "confidence": 0.706887
    }
  ]
}
